## 路径还原-模糊匹配

In [187]:
submit_id = "vague-ill3model-ill4path-ill1deduct"
sols = split.(strip.(split(strip(read(open("predict/predict_data_0708.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";

# 返回结果
res = Vector{String}(undef, 1500)
fails, notgood = Int[], Int[]
## 精确匹配
for (i, sol) in enumerate(sols)
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ques_ner[i][2]
    # 精确路径
    submits = precise_submit(ner, rels, illsdata)
    isempty(submits) && (push!(fails, i); res[i]=noans; continue)
    bettersubmits = filter(contains(rawner), submits)
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
end
println(length(fails), '\t', length(notgood))

425	8


In [193]:
# 训练集已有关系
dict_trainrels = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (que, _, rels...) in mt_train_ques_rels
    Tuple(rels) ∈ dict_trainrels[que] && continue
    push!(dict_trainrels[que], Tuple(rels))
end

In [214]:
## 剩下部分相似匹配 + 路径推理
failagain = Int[]
for i in fails
    rawner = ill_valid_ners[i]
    que = findbest(ill_train_ques, ill_valid_ques[i])
    submits = vcat([vague_submit(ner, rels, ill_triples, dict_triples_rev)
        for rels in dict_trainrels[que]]...)
    isempty(submits) && (push!(failagain, i); res[i]=noans; continue)
    bettersubmits = filter(contains(rawner), submits)
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
    print(i,'\t')
    flush(stdout)
end
length(failagain)

7	66	232	318	321	394	433	486	489	501	511	541	545	546	570	578	585	597	606	610	612	624	632	634	637	639	663	689	691	706	715	726	727	730	760	971	984	989	991	994	995	996	998	999	1133	1171	1186	1216	1218	1231	1232	1235	1245	1249	1264	1270	1282	1290	1294	1297	1303	1313	1315	1324	1331	1348	1350	1359	1361	1363	1391	1393	1396	1415	1452	1458	1459	1460	1465	1468	1487	1492	1495	

342

In [215]:
open("submit/submit_$submit_id.txt", "w") do io
    println(io, "id\tans_path")
    for (i, sol) in enumerate(res)
        println(io, i-1, '\t', sol)
    end
end

In [213]:
function dist(target, word; char=true)
    char || (words = equivills[word])
    target, word = lowercase.([target, word])
    char || (target = split(target, '_'); word = split(word, '_'))
    if char || isempty(words) # 问题匹配
        all(∉(Set(target)), word) && return typemax(Int)
        evaluate(Levenshtein(), target, word)
    else
        # 实体匹配
        words = split.(lowercase.(words), '_')
        all(all(∉(Set(target)), word) for word in words) && return typemax(Int)
        minimum(evaluate(Levenshtein(), target, word) for word in words)
    end
end

dist (generic function with 1 method)

In [ ]:
### 模糊路径 ###
"""
    find_vague_paths(ner, rels, triples; best=2)

推理对齐实体
"""
function find_vague_paths(ner, rels, triples; best = 2)
    # 跳数不够
    length(rels) == 1 && return [] # 只有一跳
    # 第一跳精确匹配
    tri1s = filter(i->i[1]==ner && i[2]==rels[1], triples)
    # isempty(tri1s) && return [] # 找不到，返回无解
    isempty(tri1s) && (tri1s = filter(i->i[1]==ner, triples)) # 找不到，更换关系
    tri1 = tri1s[1] # 随机取一个
    # 第二跳模糊匹配
    possible = filter(i->i[2]==rels[2], triples)
    # 字符匹配最佳的 best 个
    tri2s = possible[nearby(first.(possible), tri1[3]; char=false, best=best, index=true)]
    length(rels) == 2 && return [[tri1, tri] for tri in tri2s]
    # 第三跳模糊匹配
    possible = filter(i->i[2]==rels[3], triples)
    paths = []
    for tri2 in tri2s
        # 只考虑最近的两个
        for tri3 in possible[nearby(first.(possible), tri2[3], char=false, best=2, index=true)]
            push!(paths, [tri1, tri2, tri3])
        end
    end
    paths
end

"""提交模糊匹配"""
vague_submit(ner, rels, triples, dict;best=2) = submit_format.(
    vcat(valid_path.(find_vague_paths(ner, rels, triples;best=best), Ref(dict))...))

In [2]:
using StringDistances
using DataStructures

In [1]:
include("../src/extractdata.jl")
include("../src/translatedata.jl")
# include("../src/submit.jl")
# include("../src/deduction.jl")

LoadError: LoadError: UndefVarError: readtuples not defined
in expression starting at /home/rex/work_space/7 others/ccks/CCKS-mKGQA/src/extractdata.jl:6

1. `valid_submit` 寻找精确匹配的结果
2. `valid_path` 筛选可行路径，返回跨语言信息，结合 `submit_format` 提交结果


In [ ]:
println.(unique(filter(i->',' in i[1], train_ques_sols)))

SubString{String}["which format does the cause of NER, a japanese biological warfare plan during world war ii, belong to?", "night_sakura", "result", "format"]
SubString{String}["which mountain range does the client of NER, a skyscraper in london, belong to?", "lots_road_south_tower", "client", "parent"]
SubString{String}["what is the official language of the country that is affected by NER, an erdbeben?", "1957_mongolia_earthquake", "countriesaffected", "officialtext"]
SubString{String}["who leads the country that is affected by NER, an earthquake which devastated cities on the west and south coasts of anatolia (262)?", "262_southwest_anatolia_earthquake", "countriesaffected", "leadername"]
SubString{String}["who gave birth to the parent of NER, a greek mythological figure?", "telegonos", "parents", "parents"]
SubString{String}["where is the client of NER, a palace in prague, czech republic, born?", "wallenstein_palace", "client", "birthplace"]
SubString{String}["where is the client o

Excessive output truncated after 524288 bytes.

_time", "chronology", "label"]
SubString{String}["do you know what eponymous type of NER, a crater on mercury?", "ovid_(crater)", "eponym", "genre"]
SubString{String}["where is the city of Alma mater of NER, a russian linguist?", "margarita_kozhina", "almamater", "city"]
SubString{String}["do you know which south is in NER's largest city, a county in pennsylvania, united states?", "huntington_county", "largestcity", "south"]
SubString{String}["do you know where the NER name was, a 2005 hurricane?", "meteorological_history_of_hurricane_wilma", "name", "areas"]
SubString{String}["what is the profession of Election after NER, a wahl?", "dutch_general_election,_1986", "afterelection", "occupation"]
SubString{String}["who is the president of NER's Alma mater, a Taiwanese actor and television presenter?", "lin_tzay-peir", "almamater", "president"]
SubString{String}["who is the developer of NER Things, head-mounted optical computer glasses?", "google_glass", "related", "developer"]
SubString{

4196-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [ ]:
tri1 = filter(i->i[1]==ner && i[2]==rel1, triples)[1]

In [ ]:
possible = unique!(first.(filter(i->i[2]==rel2, triples)))
res, scos = nearby(possible, tri1[end]; char=false)
ner2 = res[1]
filter(i->i[1]==ner2 && i[2]==rel2, triples)[1]

In [ ]:
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (sub, rel, obj) in triples
    push!(edges[sub], (rel, obj))
end

firstrel (generic function with 1 method)

In [ ]:
fails = filter(∉(Set(res)), valid_ques)

135-element Vector{String}:
 "what is involved in the important events attended by NER?"
 "which house does the event of N" ⋯ 81 bytes ⋯ "to the 11th century, belong to?"
 "where did the combatant of NER die?"
 "which place is called as the subdivision of the city in the south of NER?"
 "which national team does NER's coach belong to?"
 "who is before the nickname of NER?"
 "who is the predecessor of the c" ⋯ 24 bytes ⋯ "al building in corleone, italy?"
 "what is the club that NER's head coach belongs to?"
 "where is the city in the south of NER seated?"
 "which sport does NER's coach play?"
 "which place is called as the subdivision of the southeast of the south of NER?"
 "which timezone does the north of the west of NER belong to?"
 "which dynasty does the legislature of the state party of NER belong to?"
 ⋮
 "who is the heir to the front of NER's title"
 "do You Know what Language Lyrics Of NER's Title Is Using"
 "do you know who is the predecessor of The name of the ship of NER"
 "

In [ ]:
que, ner = valid_ques_ner[4]
ques, scores = top10(que)
rels = vcat([train_rels[que] for que in ques]...)
first.(collect(sort_count(counter(first.(rels)))))

3-element Vector{SubString{String}}:
 "parents"
 "theme"
 "afterelection"